In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
#import pygplates
#import pygmt

#from scipy.stats import median_abs_deviation

import sys
sys.path.append('../python/')
import joyful_geochemistry as joy
#import joyful_mapping as joymap

%matplotlib inline
%load_ext autoreload
%autoreload 2



In [ ]:
# For the Luffi and Ducea calibration, we need to point the code to the csv files containing the lookup tables
model_dir = '../luffi/REM_surfaces_csv/'
gc_interpolator_dict = joy.make_gc_interpolator_dict(model_dir)


# Load the data. 
# Note, all code assumes a specific labelling convention that mirrors the names used
# in the Gard et al (2019) global database
#df = joy.geochem_from_csv('../datafiles/geochem_merge_20221026.csv',
#                          longitude_field_name='Longitude', latitude_field_name='Latitude')
df = joy.geochem_from_csv('../../global_paper/datafiles/geochem_global_20230124_subduction_M2021.csv',
                          longitude_field_name='longitude', latitude_field_name='latitude')


# This step would filter to take only the samples within the Andean Cordillera according
# to the polygons of Hasterok et al. But needs to have GPlatesReconstructionModel module available
#''' 
import joyful_mapping as joymap
df = joymap.select_orogens(df,gdf=None, 
               orogen_names='Cordilleran', 
               continent_names='South America',
               region=[-100, -50, -60, 20])
#'''

df['a'].hist()


In [ ]:
# Example 1: Get elevations or La/Yb only

calibration = 'luffi'
mohometer_selection = ['la_yb_elevation']

# Filter the database according to the criteria listed in the paper, and optionally isolating 
# data within a specific time window
df_filt = joy.filter_the_database(df, calibration, 
                                  age_min=5, age_max=350)

# Use calibrations to get elevation estimates from geochemical values
elevations_df = joy.get_elevations(df_filt, 
                                   gc_interpolator_dict=gc_interpolator_dict,
                                   calibration=calibration,
                                   mohometer_selection=mohometer_selection)


# link the results back to the coordinates and age information
elevations_df = gpd.GeoDataFrame(elevations_df, geometry=df_filt.geometry, crs=4326).join(df_filt['age'])

elevations_df


In [ ]:
# Example 2: Get elevations for all 41 mohometers
calibration = 'luffi'
mohometer_selection = 41

# Filter the database according to the criteria listed in the paper
df_filt = joy.filter_the_database(df, calibration, 
                                  age_min=0, age_max=410)

# Use calibrations to get elevation estimates from geochemical values
elevations_df = joy.get_elevations(df_filt, 
                                   gc_interpolator_dict=gc_interpolator_dict,
                                   calibration=calibration,
                                   mohometer_selection=mohometer_selection)


# link the results back to the coordinates and age information
elevations_df = gpd.GeoDataFrame(elevations_df, geometry=df_filt.geometry, crs=4326).join(df_filt['age'])

# write to a file for Satyam
elevations_df['Longitude'] = elevations_df.geometry.x
elevations_df['Latitude'] = elevations_df.geometry.y
elevations_df.drop(columns='geometry').to_csv('./paleoelevations_for_satyam.csv', index=False)


In [ ]:
# Example 3: Get elevations for all 41 mohometers, and get the median at each sample location

calibration = 'luffi'
mohometer_selection = 41

# Filter the database according to the criteria listed in the paper
df_filt = joy.filter_the_database(df, calibration, 
                                  age_min=0, age_max=350)

# Use calibrations to get elevation estimates from geochemical values
elevations_df = joy.get_elevations(df_filt, 
                                   gc_interpolator_dict=gc_interpolator_dict,
                                   calibration=calibration,
                                   mohometer_selection=mohometer_selection)

elevations_df = elevations_df.median(axis=1)

# link the results back to the coordinates and age information
elevations_df = gpd.GeoDataFrame(elevations_df, geometry=df_filt.geometry, crs=4326).join(df_filt['age'])



In [ ]:
fig,ax = plt.subplots(figsize=(12,20))
elevations_df.plot(column=0, ax=ax, vmin=0, vmax=5000, legend=True)
#plt.colorbar()

In [ ]:
elevations_df